In [1]:
import numpy as np
import cv2
import pandas as pd

np.random.seed(42)
import numpy as np
import cv2
import pandas as pd
import imghdr

np.random.seed(42)
class Yolo_BBox_Visualizer:

    def __init__(self):
        pass

    def read_txt(self, path_to_txt):
        df = pd.read_csv(f"{path_to_txt}",
                         sep=" ",
                         names=["label", "x", "y", "w", "h"])
        return df

    def read_classes(self, classes_file_path, random=True):
        with open(classes_file_path, "r") as f:
            self.classes_list = f.read().splitlines()
        # Random Colors list
        if random:
            self.color_list = np.random.uniform(
                low=0,
                high=255,
                size=(len(self.classes_list), 3)
                )
        # Green color only
        else:
            classes = np.zeros(len(self.classes_list))
            self.color_list = [(0,255,0) for i in classes]


    def convert(self, size, x, y, w, h):
        """
        From https://github.com/adi1999/yolo-bbox-visualizer/blob/master/main.py
        Convert bounding box coordinates
        size: tuple representing the width and height of an image
        x: x-coordinate of the center of a bounding box
        y: y-coordinate of the center of a bounding box
        w: width of the bounding box in fractional representation
        h: height of the bounding box
        return 
        """
        box = np.zeros(4)
        dw = 1. / size[0]
        dh = 1. / size[1]
        x = x / dw
        w = w / dw
        y = y / dh
        h = h / dh
        box[0] = x - (w / 2.0)
        box[1] = x + (w / 2.0)
        box[2] = y - (h / 2.0)
        box[3] = y + (h / 2.0)

        return (box)

    def draw_bbox(self, image, df):
        class_color = self.color_list
        for i, row in df.iterrows():
            
            x, y, w, h =row['x'], row['y'], row['w'], row['h']
            img_h, img_w = image.shape[:2]
            bb = self.convert((img_w, img_h), x, y, w, h)
            cv2.rectangle(image, (int(round(bb[0])), int(round(bb[2]))),
                          (int(round(bb[1])), int(round(bb[3]))), class_color[int(row["label"])],
                          1)
        return image

    def draw_image(self, image_path, path_to_txt, img_name = "Image.jpg"):
        image = cv2.imread(image_path)
        bbox_image = self.draw_bbox(image, self.read_txt(path_to_txt))

        if image is None:
            print("Error: Failed to read image from file")
            return
        try:
            cv2.imwrite(img_name, bbox_image)
            cv2.imshow("Result", bbox_image)
            cv2.waitKey(0)

            cv2.destroyAllWindows()
        except:
            print("Error")
            

In [2]:
img_path = 'img.jpg'
label_path ='label.txt'

In [3]:
class_file = "class.names"

est = Yolo_BBox_Visualizer()
est.read_classes(class_file, random=True)
est.draw_image(img_path, label_path)